In [1]:
import tkinter as tk
import mysql.connector as m
from datetime import date
global tname,tdob,tnum,tmail,edit
global ename,edob,enum
import face_recognition
import cv2
from datetime import datetime
import mysql.connector
import numpy as np
from PIL import Image,ImageTk
import tkcalendar as tc
from tkinter.filedialog import askopenfilename
from tkcalendar import Calendar,DateEntry
from tkinter import messagebox
def update(id,w):
    d=Tk()
    mydb=mysql.connector.connect(host="localhost",user="root1",password="ramprakashpg01",database="register")
    mycursor=mydb.cursor()
    print(id)
    mycursor.execute("SELECT name,contact,email FROM student1 WHERE id='%d'" %(id))
    mydb.commit()
    myu=mycursor.fetchall()
    for i in myu:
        d.minsize(500,500)
        d.maxsize(500,500)
        tk.Label(d,text="Student's name",bg="orange",font=("Consolas",20)).place(x=50,y=170)
        tk.Label(d,text="Parent number",bg="orange",font=("Consolas",20)).place(x=50,y=210)
        tk.Label(d,text="Email-ID",bg="orange",font=("Consolas",20)).place(x=50,y=250)
        a=tk.Entry(d,textvariable=tname).place(x=290,y=180).insert(END,i[0])
        b=tk.Entry(d,textvariable=tnum).place(x=290,y=220)
        c=tk.Entry(d,textvariable=tmail).place(x=290,y=260)
def delete(id,tk):
    print(id)
    mycursor.execute("delete from student1 where id='%d'" % (id))
    mydb.commit()
    tk.destroy()
    data()
def data():
    mydb=mysql.connector.connect(host="localhost",user="root1",password="ramprakashpg01",database="register")
    mycursor=mydb.cursor()
    root=Tk()
    root.minsize(800,500)
    root.title("DATABASE")
    root.maxsize(800,500)
    root.configure(bg="#3b5998")
    mycursor.execute("SELECT id,name,email,contact FROM student1")
    my=mycursor.fetchall()
    print(my)
    j=2
    Label(root,text="DATABASE VIEW",font=("Consolas",25),bg="#3b5998",fg="white").grid(row=0,column=50)
    Label(root,text="Id",bg="#3b5998",font=("Consolas",25),fg="white").grid(row=1,column=0)
    Label(root,text="Name",bg="#3b5998",font=("Consolas",25),fg="white").grid(row=1,column=25)
    Label(root,text="Email",bg="#3b5998",font=("Consolas",25),fg="white").grid(row=1,column=50)
    Label(root,text="Phone",bg="#3b5998",font=("Consolas",25),fg="white").grid(row=1,column=65)
    for i in my:
        Label(root,text=i[0],font=("Consolas",15),bg="#3b5998",fg="white").grid(row=j,column=0)
        Label(root,text=i[1],font=("Consolas",15),bg="#3b5998",fg="white").grid(row=j,column=25)
        Label(root,text=i[2],font=("Consolas",15),bg="#3b5998",fg="white").grid(row=j,column=50)
        Label(root,text=i[3],font=("Consolas",15),bg="#3b5998",fg="white").grid(row=j,column=65)
        Button(root,text="Delete",command=lambda:delete(i[0],root)).grid(row=j,column=100)
        Button(root,text="Update",command=lambda:update(i[0],root)).grid(row=j,column=75)
        j+=1
def face(w):
    w.destroy()
    mydb=mysql.connector.connect(host="localhost",user="root1",password="ramprakashpg01",database="register")
    mycursor=mydb.cursor()
    known_face_names=[]
    known_face_encodings=[]
    video_capture = cv2.VideoCapture(0)
    mycursor.execute("SELECT id,image FROM student1")
    mycom=mycursor.fetchall()
    print(mycom)
    for i in mycom:
        print(i[0])
        image = face_recognition.load_image_file(i[1])
        image_face_encoding = face_recognition.face_encodings(image)[0]  
        known_face_names.append(str(i[0]))
        known_face_encodings.append(image_face_encoding)
        print("done")
    face_locations = []
    face_encodings = []
    face_names = []
    process_this_frame = True
    c=0
    while True:
        ret,frame=video_capture.read()
        small_frame = cv2.resize(frame,(0,0), fx=0.25, fy=0.25)

        rgb_small_frame = small_frame[:, :, ::-1]

        if process_this_frame:
            face_locations = face_recognition.face_locations(rgb_small_frame) #enmc
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
            face_names = []
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"

                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]
                face_names.append(name)
                if (name!="Unknown"):
                    mycursor.execute("""UPDATE student1 SET mark=1,total=total+1 WHERE mark=0 AND id=%s"""%name)
                    mydb.commit()
                    c=c+1
                    if c==1:
                        mycursor.execute("INSERT INTO student2 (id,date,time) VALUES(%s,%s,%s)",(int(name),date.today(),datetime.now()))
                        mydb.commit()
                else:
                    print("Unknown Name")
        process_this_frame = not process_this_frame
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        cv2.imshow('Attendance Survillence', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    video_capture.release()
    cv2.destroyAllWindows()
def img():
    file1=askopenfilename()
    img=Image.open(file1)
    img=img.resize((100,100),Image.ANTIALIAS)
    tkimage=ImageTk.PhotoImage(img)
    myvar=tk.Label(root,image=tkimage).place(x=298,y=300)
    l1=root.tk.splitlist(file1)
    print(l1)
    tk.Button(root,text="Register",command=lambda:check(l1),font=("Consolas",15)).place(x=200,y=450)
    myvar.image=tkimage
def check(l1):
    if((tid.get()=='')or(tname.get()=='')or(tnum.get()=='')):
        messagebox.showerror(title="Error",message="Fill every column properly...")
    print(l1[0])
    mysql=m.connect(host='localhost',user='root1',password='ramprakashpg01',database='register')
    mycursor=mysql.cursor()
    print(tnum.get())
    mycursor.execute('insert into student1(id,name,mark,image,total,contact,email) values(%s,%s,%s,%s,%s,%s,%s)',(int(tid.get()),tname.get(),0,l1[0],0,'+91'+tnum.get(),tmail.get()))
    mysql.commit()
    messagebox.showinfo(title="Success",message="Data Uploaded")
root=tk.Tk()
root.title("Registration form")
root.configure(bg="orange")
root.minsize(500,500)
root.maxsize(500,500)
tname=tk.StringVar()
tmail=tk.StringVar()
tnum=tk.StringVar()
tdob=tk.StringVar()
tid=tk.StringVar()
title=tk.Label(root,text='ABC School',bg="orange",font=("Consolas",35)).place(x=100,y=0)
content=tk.Label(root,text='Register new student',bg="orange",font=("Consolas",25)).place(x=75,y=50)
name=tk.Label(root,text="Student's name",bg="orange",font=("Consolas",20)).place(x=50,y=170)
num=tk.Label(root,text="Parent number",bg="orange",font=("Consolas",20)).place(x=50,y=210)
tk.Label(root,text="Email-ID",bg="orange",font=("Consolas",20)).place(x=50,y=250)
id=tk.Label(root,text="id",bg="orange",font=("Consolas",20)).place(x=50,y=130)
tk.Label(root,text="Photo",bg="orange",font=("Consolas",20)).place(x=50,y=290)
Button(root,text="Database",bg="orange",font=("Consolas",15),command=lambda:data()).place(x=20,y=360)
ename=tk.Entry(root,textvariable=tname).place(x=290,y=180)
enum=tk.Entry(root,textvariable=tnum).place(x=290,y=220)
email=tk.Entry(root,textvariable=tmail).place(x=290,y=260)
eid=tk.Entry(root,textvariable=tid).place(x=290,y=140)
but=tk.Button(root,text="Choose file",font=("Consolas",15),command=img).place(x=150,y=300)
tk.Button(root,text="Go to page-->",command=lambda:face(root)).place(x=20,y=330)
root.mainloop()

ModuleNotFoundError: No module named 'face_recognition'